In [1]:
import os
import getpass
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

In [2]:
%load_ext dotenv
%dotenv -o ../.env

In [3]:
loader = DirectoryLoader("./examples", glob="**/*.tf", loader_cls=TextLoader, show_progress=True)
raw_documents = loader.load()
raw_documents = raw_documents[:10]
raw_documents

100%|██████████| 182/182 [00:00<00:00, 7063.80it/s]


[Document(page_content='# Amazon SNS\n\nresource "aws_sns_topic" "user_updates" {\n  name            = "user-updates-topic"\n  delivery_policy = <<EOF\n{\n  "http": {\n    "defaultHealthyRetryPolicy": {\n      "minDelayTarget": 20,\n      "maxDelayTarget": 20,\n      "numRetries": 3,\n      "numMaxDelayRetries": 0,\n      "numNoDelayRetries": 0,\n      "numMinDelayRetries": 0,\n      "backoffFunction": "linear"\n    },\n    "disableSubscriptionOverrides": false,\n    "defaultThrottlePolicy": {\n      "maxReceivesPerSecond": 1\n    }\n  }\n}\nEOF\n}', metadata={'source': 'examples/sns.tf'}),
 Document(page_content='resource "aws_ecs_service" "mongo" {\n  name            = "mongodb"\n  cluster         = aws_ecs_cluster.foo.id\n  task_definition = aws_ecs_task_definition.mongo.arn\n  desired_count   = 3\n  iam_role        = aws_iam_role.foo.arn\n  depends_on      = [aws_iam_role_policy.foo]\n\n  ordered_placement_strategy {\n    type  = "binpack"\n    field = "cpu"\n  }\n\n  load_balancer

In [4]:
db = Chroma.from_documents(raw_documents, OpenAIEmbeddings(), persist_directory="./chroma_db")

/Users/malte/miniconda3/envs/infragen/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [5]:
query = "RDS"
docs = db.similarity_search(query)
docs


[Document(page_content='# Amazon RDS example\n\nmodule "db" {\n  source = "terraform-aws-modules/rds/aws"\n\n  identifier = "demodb"\n\n  engine            = "mysql"\n  engine_version    = "5.7"\n  instance_class    = "db.t3a.large"\n  allocated_storage = 5\n\n  db_name  = "demodb"\n  username = "user"\n  port     = "3306"\n\n  iam_database_authentication_enabled = true\n\n  vpc_security_group_ids = ["sg-12345678"]\n\n  maintenance_window = "Mon:00:00-Mon:03:00"\n  backup_window      = "03:00-06:00"\n\n  # Enhanced Monitoring - see example for details on how to create the role\n  # by yourself, in case you don\'t want to create it automatically\n  monitoring_interval    = "30"\n  monitoring_role_name   = "MyRDSMonitoringRole"\n  create_monitoring_role = true\n\n  tags = {\n    Owner       = "user"\n    Environment = "dev"\n  }\n\n  # DB subnet group\n  create_db_subnet_group = true\n  subnet_ids             = ["subnet-12345678", "subnet-87654321"]\n\n  # DB parameter group\n  family = 

In [7]:
# get documents
from langchain.vectorstores.chroma import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

db = Chroma(embedding_function=OpenAIEmbeddings(), persist_directory="./chroma_db")



[(Document(page_content='# Amazon RDS example\n\nmodule "db" {\n  source = "terraform-aws-modules/rds/aws"\n\n  identifier = "demodb"\n\n  engine            = "mysql"\n  engine_version    = "5.7"\n  instance_class    = "db.t3a.large"\n  allocated_storage = 5\n\n  db_name  = "demodb"\n  username = "user"\n  port     = "3306"\n\n  iam_database_authentication_enabled = true\n\n  vpc_security_group_ids = ["sg-12345678"]\n\n  maintenance_window = "Mon:00:00-Mon:03:00"\n  backup_window      = "03:00-06:00"\n\n  # Enhanced Monitoring - see example for details on how to create the role\n  # by yourself, in case you don\'t want to create it automatically\n  monitoring_interval    = "30"\n  monitoring_role_name   = "MyRDSMonitoringRole"\n  create_monitoring_role = true\n\n  tags = {\n    Owner       = "user"\n    Environment = "dev"\n  }\n\n  # DB subnet group\n  create_db_subnet_group = true\n  subnet_ids             = ["subnet-12345678", "subnet-87654321"]\n\n  # DB parameter group\n  family =

In [14]:
db.similarity_search("RDS", k=1)

[Document(page_content='# Amazon RDS example\n\nmodule "db" {\n  source = "terraform-aws-modules/rds/aws"\n\n  identifier = "demodb"\n\n  engine            = "mysql"\n  engine_version    = "5.7"\n  instance_class    = "db.t3a.large"\n  allocated_storage = 5\n\n  db_name  = "demodb"\n  username = "user"\n  port     = "3306"\n\n  iam_database_authentication_enabled = true\n\n  vpc_security_group_ids = ["sg-12345678"]\n\n  maintenance_window = "Mon:00:00-Mon:03:00"\n  backup_window      = "03:00-06:00"\n\n  # Enhanced Monitoring - see example for details on how to create the role\n  # by yourself, in case you don\'t want to create it automatically\n  monitoring_interval    = "30"\n  monitoring_role_name   = "MyRDSMonitoringRole"\n  create_monitoring_role = true\n\n  tags = {\n    Owner       = "user"\n    Environment = "dev"\n  }\n\n  # DB subnet group\n  create_db_subnet_group = true\n  subnet_ids             = ["subnet-12345678", "subnet-87654321"]\n\n  # DB parameter group\n  family = 